In [1]:
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd 


In [2]:
fields = ['transport','data','cloud']

In [3]:
# need to build list of cities
# iterate each cities with get requests
soups = []

# need a way to check pagination container for list length, thats how we know how long it is
for field in fields:
    URL = 'https://www.postgrad.com/search/chevening/?q='+ field + '&uk_location='
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    soups.append(soup)

In [4]:
#use the initial html soup scraped to figure out how many pages each query has
pages_list = []

for soup in soups:
    page_container = soup.find('div', class_='pagination-container')
    pages = page_container.find_all('li')

    #number of of <li> indicates number of pages. -1 because 1 <li> represents the active page
    if len(pages) != 1:
        pages_list.append(len(pages)-1)
    else:
        pages_list.append(1)

In [112]:
pages_list

[2, 8, 1]

In [146]:
field_page_dict = dict(zip(fields, pages_list))
new_soups = []
for field,pages in field_page_dict.items():
    for page in range(1,pages+1):
        URL = 'https://www.postgrad.com/search/chevening/?q='+ field + '&uk_location=&page=' + str(page)
        print(URL)
        page = requests.get(URL)
        new_soup = BeautifulSoup(page.content, 'html.parser')
        new_soups.append(new_soup)

https://www.postgrad.com/search/chevening/?q=transport&uk_location=&page=1
https://www.postgrad.com/search/chevening/?q=transport&uk_location=&page=2
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=1
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=2
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=3
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=4
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=5
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=6
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=7
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=8
https://www.postgrad.com/search/chevening/?q=cloud&uk_location=&page=1


In [144]:
len(new_soups)

11

In [148]:
sections_list =[]

for new_soup in new_soups:
    sections = new_soup.find_all('section', class_='search_dept')
    sections_list.append(sections)


In [151]:
def dataFramer(sections):
    unis = []
    courses = []
    chevenings = []
    for section in sections:

        uni = section.find('div', class_='search_dept-head')
        #Try to find unis that is are chevening
        try :
            uni.find('a', class_='tag tag-tick').text
            chevening = 'True'
            uni = uni.find_all('a')
            uni = uni[1].text

        except:
            chevening = 'False'
            uni = uni.find('a').text 

        #Try to find unis with more than one course

        course_list = section.find_all('li')
        for course in course_list:
            course  = course.find('a', class_= 'course-name').text
            courses.append(course)
            unis.append(uni)
            chevenings.append(chevening)
        #courses.append(course.find('a', class_= 'course-name').text)

    df = pd.DataFrame(list(zip(unis, courses,chevenings)),
                columns =['University', 'Courses','Chevening'])
    return df

In [175]:
#Using dataFramer function on all sections that and all data to be appended to dataframe
dataframe_list = []
for sections in sections_list:
    section_data = dataFramer(sections)
    dataframe_list.append(section_data)
all_data =pd.concat(dataframe_list)

In [179]:
all_data
all_data.to_csv('chevening_university.csv',index=False)